In [2]:
import torch
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

Using PyTorch version: 2.6.0+cu124  Device: cuda


In [2]:
pip install torch

^C
Note: you may need to restart the kernel to use updated packages.


   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
   --- ----------------------------------- 21.2/216.1 MB 103.3 MB/s eta 0:00:02
   ------- ------------------------------- 41.9/216.1 MB 106.9 MB/s eta 0:00:02
   --------- ------------------------------ 48.8/216.1 MB 77.6 MB/s eta 0:00:03
   ------------ --------------------------- 68.2/216.1 MB 82.0 MB/s eta 0:00:02
   -------------- ------------------------- 77.6/216.1 MB 75.0 MB/s eta 0:00:02
   ----------------- ---------------------- 96.7/216.1 MB 78.2 MB/s eta 0:00:02
   ------------------- ------------------- 110.4/216.1 MB 76.6 MB/s eta 0:00:02
   -------------------- ------------------ 111.7/216.1 MB 68.6 MB/s eta 0:00:02
   ---------------------- ---------------- 126.9/216.1 MB 68.1 MB/s eta 0:00:02
   ------------------------ -------------- 135.3/216.1 MB 66.5 MB/s eta 0:00:02
   ------------------------ -------------- 135.5/216.1 MB 62.3 MB/s eta 0:00:02
   --------------------------- ----------- 152.3/

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import time
import os


ModuleNotFoundError: No module named 'torch'

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [5]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transforms['train'])
val_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transforms['val'])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

class_names = train_dataset.classes

100%|██████████| 170M/170M [00:04<00:00, 37.6MB/s] 


In [6]:
model_transfer = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 215MB/s] 


In [ ]:
from torchinfo import summary

In [ ]:
summary(model_transfer)

In [7]:
for param in model_transfer.features.parameters():
    #print(param)
    param.requires_grad = False

In [8]:
len(class_names)

10

In [9]:
model_transfer.classifier[6] = nn.Linear(model_transfer.classifier[6].in_features, len(class_names))

In [10]:
model_transfer.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=10, bias=True)
)

In [16]:
model_transfer = model_transfer.to(DEVICE)
optimizer  = optim.Adam(model_transfer.classifier.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [12]:
for param in model_transfer.classifier.parameters():
    print(param.requires_grad)

True
True
True
True
True
True


In [13]:

def train(model, train_loader, optimizer, log_interval):
    global output, label
    model.train()
    correct = 0
    train_loss = 0
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()


        _, prediction = torch.max(output, dim=1 )
        # 정답을 맞춘 갯수
        # acc += sum(idx == label)
        correct += prediction.eq(label.view_as(prediction)).sum().item()


        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image),
                len(train_loader.dataset), 100. * batch_idx / len(train_loader),
                loss.item()))
           
    train_loss /= (len(train_loader.dataset) / BATCH_SIZE)
    train_accuracy = 100. * correct / len(train_loader.dataset)


    return train_loss, train_accuracy



In [14]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0


    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()


    test_loss /= (len(test_loader.dataset) / BATCH_SIZE)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy


In [17]:
EPOCHS = 5
BATCH_SIZE = 32
loss_hist_train     = [0] * EPOCHS
accuracy_hist_train = [0] * EPOCHS
loss_hist_valid     = [0] * EPOCHS
accuracy_hist_valid = [0] * EPOCHS

for epoch in range(1, EPOCHS + 1):
    loss_, acc_ = train(model_transfer, train_loader, optimizer, log_interval = 200)
    loss_hist_train[epoch] = loss_
    accuracy_hist_train[epoch] = acc_
    test_loss, test_accuracy = evaluate(model_transfer, val_loader)
    loss_hist_valid[epoch] = test_loss
    accuracy_hist_valid[epoch] = test_accuracy
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))



Train Epoch: 1 [0/50000 (0%)]	Train Loss: 2.381358
Train Epoch: 1 [6400/50000 (13%)]	Train Loss: 1.471007
Train Epoch: 1 [12800/50000 (26%)]	Train Loss: 1.186061
Train Epoch: 1 [19200/50000 (38%)]	Train Loss: 0.530141
Train Epoch: 1 [25600/50000 (51%)]	Train Loss: 0.776385
Train Epoch: 1 [32000/50000 (64%)]	Train Loss: 1.576459
Train Epoch: 1 [38400/50000 (77%)]	Train Loss: 0.810197
Train Epoch: 1 [44800/50000 (90%)]	Train Loss: 1.009310

[EPOCH: 1], 	Test Loss: 0.5124, 	Test Accuracy: 84.03 % 

Train Epoch: 2 [0/50000 (0%)]	Train Loss: 0.795635


KeyboardInterrupt: 